In [ ]:
# Telecom Revenue Assurance AI Model (Custom-Tuned Bert)
# Author: Fatih E. NAR

In [ ]:
# Install Necessary Libraries
!pip install -r requirements.txt
!pip install transformers datasets torch scikit-learn accelerate -U

In [ ]:
# Import Libraries
import pandas as pd
import torch
import numpy as np
import lzma
import shutil
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Extract the .xz file
with lzma.open('data/telecom_revass_data.csv.xz', 'rb') as f_in:
    with open('data/telecom_revass_data.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Check if CUDA (GPU) is available and set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load your dataset
df = pd.read_csv('data/telecom_revass_data.csv')

# Convert data to text format (example)
df['text'] = df.apply(lambda row: f"Call Duration: {row['Call_Duration']}, Data Usage: {row['Data_Usage']}, SMS Count: {row['Sms_Count']}, Roaming Indicator: {row['Roaming_Indicator']}, Mobile Wallet Use: {row['MobileWallet_Use']}, Plan Type: {row['Plan_Type']}, Cost: {row['Cost']}, Cellular Location Distance: {row['Cellular_Location_Distance']}, Last Time Pin Used: {row['Last_Time_Pin_Used']}, Avg Call Duration: {row['Avg_Call_Duration']}, Avg Data Usage: {row['Avg_Data_Usage']}, Avg Cost: {row['Avg_Cost']}", axis=1)

# Split the data
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['Fraud'])

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['text', 'Fraud']])
test_dataset = Dataset.from_pandas(test_df[['text', 'Fraud']])

In [ ]:
# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'  # Example placeholder, replace with actual PaLM model if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column('Fraud', 'labels')
test_dataset = test_dataset.rename_column('Fraud', 'labels')

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=10,
)

# Define custom compute_metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = (pred == labels).mean()
    return {'accuracy': accuracy}

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

In [ ]:
# Save the fine-tuned model
model.save_pretrained('models/fine-tuned-bert-revass')
tokenizer.save_pretrained('models/fine-tuned-bert-revass')